参数的微小变动随着网络的深度增加而放大, BN通过改变输入的分布进而减少网络内部的协变量偏移, 有一定的正则效果

# BatchNormalization
- 减少了对参数初始化的依赖
- 能以更大的学习率学习
- 一些场合能避免使用Dropout
+ 加速网络收敛

$SGD通过优化网络参数\Theta来得到最小的Loss$
\begin{equation}
\Theta = \mathop \arg\min_{\Theta} \frac{1}{N}\sum_{i=1}^{N}l(x_i, \Theta)
\end{equation}

对于内部网络计算

$l=F_2(F_1(u,\Theta_1),\Theta_2)$

$\Theta_2的学习可等效为l=F_2(x,\Theta_2)$

$输入x=F_1(u,\Theta_1)的分布对\Theta_2的学习至关重要$

早期的sigmoid函数在输入到达S曲线的顶部时梯度为零,使网络的训练需要将参数约束到S曲线的上升区间内,造成BP的梯度消失, 而这样的结果,使得输入的大多数维度进入饱和状态,延长了网络收敛.

随后出现的应对方法是ReLu, Glorot/Xavier初始化, 使用较小小的学习率,

### Reducing Internal Covariate Shift
输入白化后网络训练能更快的收敛, 线性变换至0均值,1方差, 去相关.

白化每一层的输入, 可以使输入保持一个固定的分布,来移除网络内部的协变量偏移.

如果将白化添加到BP的优化过程中, bias在中心化时会被抵消, 而梯度的不断更新会导致bias无限增大, 会导致归一化后BP的优化失效.  

对此, BP中我们需计算输入x以及对应的原始样本雅可比矩阵

$\frac{\partial Norm(x,\chi)}{\partial x}和\frac{\partial Norm(x,\chi)}{\partial \chi}$

### Normalization Statistics via Mini-Batch
完全白化每一层的输入代价非常大,而且不能保证处处可微,
两步简化
> 1. 单独归一化每一层的标量特征, 即使特征未去相关,仍然能加速收敛
$$\hat{x}^{(k)} = \frac{x^(k)-E[x^{(k)}]}{\sqrt{ Var[x^{(k)}]} }$$

然而单纯的归一化可能会改变网络的表示能力, 如sigmoid函数的归一化会丧失其部分非线性变换

通过引入额外的平移变换与尺度变换$$y^{(k)}=\gamma^{(k)}\hat{x}^{(k)}+\beta^{(k)}$$

其中使$\gamma^{(k)}=\sqrt{Var[x^{(k)}}, \beta^{(k)}=E[x^{(k)}]$能还原原始的表示能力
> 2. 取每个batch_size的均值和方差来进行BN

相较于协方差的方法, 激活函数的白化数比batch_size要大,涉及到非奇异矩阵的计算, 需要添加正则项; 仅计算每个维度的方差要简单得多

添加BN到网络中可得到任意的激活, 

$其中\gamma与\beta参数是可以通过学习得到的$
在归一化时需添加一个常数,避免batch_size内的样本方差为0而分母为0

$$\hat{x}_i \leftarrow \frac{x_i-\mu_B}{\sqrt{\sigma^2_B+\epsilon}}$$

### Training and Inference with Batch-Normalized Networks
激活函数的归一化能达到有效的训练, 而在推理的过程中是不必要的

在推理过程, 取所有Batch_size得到的均值和方差的平均, 
$E_B[\mu_B]\rightarrow E[x]$

$\frac{m}{m-1}E_B[\sigma_B^2] \rightarrow Var[x]$

将BN层的变换公式替换成
$$y=\frac{\gamma}{\sqrt{Var[x]+\epsilon}}x+(\beta-\frac{\gamma E[x]}{\sqrt{Var[x]+\epsilon}})$$

### Batch-Normalized Convolutional Networks
对网络中的z = g(Wu+b)

u通常为上层的非线性输出,其分布在训练中可能发生改变,

Wu+b更可能拥有对称,非稀疏分布, 对其进行归一化相当是给一稳定的分布进行线性激活, 其中b可以被忽略
$z=(Wu+b)\rightarrow z=(BN(Wu))$

### Batch Normalization enables higher learning rate

- 能防止训练卡住在非线性饱和区
- 对参数的尺度有更强的适应性, 能稳固参数的增长

尺度变换不会影响到网络层的雅可比矩阵